In [1]:
import numpy as np 
import pandas as pd
from PyPDF2 import PdfFileReader,PdfReader
import fitz
import re
import os
#import tabula
import time
import warnings

In [2]:
from PyPDF2.errors import PdfReadError

In [3]:
warnings.filterwarnings('ignore')

In [4]:
pd.set_option('display.width', 1000,
              'display.max_columns', 10,
              'display.max_colwidth', 100)

# Define Function to get DF for best case 

## Function to convert text of PDF file to transaction 

### Normal Case

In [7]:
def convert_pdf_to_csv(pdf_path:str,excel_path='C:/Users/trilnd/Desktop/QBO/Test_file/'):
    """
    pdf_path: the directory of PDF file. For example: 'C:/Users/trilnd/Desktop/QBO/File PDF/Citi CC.pdf'
    Kind_extract_pattern is Format of Trans date and Post date.
    Will be divide into 2 kinds:
        pattern 1: Abbreviations of Month name and 2 digits o day number (For example: Sep 12, Sep 05)
        pattern 2: Date format (dd/mm or mm/dd)
    If the user choose pattern 1: type 1. Pattern 2: type 2
    excel_name: Name the excel file which was converted from PDF file
    """  
    
    #Function to get description

    def get_description(trans,len_upper,len_lower):
        if re.search('www.53.com/businessbanking',all_text_page):
            return trans[len_upper+len_lower+1:]
        else:
        # check space: 
            check_space = pat+'(?=\s)'
            if re.search(check_space,trans) != None:
                return trans[len_upper+1:(len(trans)-len_lower)]
            else:
                return trans[(len_upper):(len(trans)-len_lower)]

    pattern_1 = r"^([A-Z][a-z]{2}\s?\d{1,2})"
    pattern_2 = r'^((\d{1,2}/\d{1,2})(/\d{2})?)'
    pattern_3 = r'^((\d{1,2}-\d{1,2})(-\d{2})?)'
    base_name = os.path.basename(pdf_path)[0:-4]
    pdf_dict = {base_name:{'Status':'',
                           'Transaction':'',
                           'Dataframe':''}}
    
    #1: Check whether the path is correct or wrong
    try:
        with open(pdf_path,'rb') as file:
            print(f"{base_name}: Connect file successful")
            pdf_dict[base_name]["Status"] = "Connect file successful"
    except IOError:
            print("Can not find the file directory or file name does not exist")
            pdf_dict[base_name]["Status"] = "Can not find the file directory or file name does not exist"
            return pdf_dict

    #2: Implement to convert file to pdf variable and print total number pages
    else:
        try:
            pdf_file = PdfReader(pdf_path,strict=True)
        except Exception as e:
            pdf_dict[base_name]["Status"] = "Error: Can not read this file"
            return pdf_dict
        else:
            number_page = pdf_file.getNumPages()
            #print("Total number pages of File PDF are",number_page)
    
        #3: Convert pdf file to the string 
            all_text_page =''
            for page_num in range(0,pdf_file.getNumPages()):
                all_text_page += (pdf_file.getPage(page_num).extractText()+'\n')
                
            all_text_list = [line.strip() for line in all_text_page.split('\n')]
            
        # : Check if the the file contain text or numeric?
            if re.search(r'.',all_text_page) is None:
                print("This PDF file is Scan data")
                pdf_dict[base_name]["Status"] = "This PDF file is Scan data"
                return pdf_dict
            
        # Select amount pattern if this PDF is AMEX or Citi or capitalone => amount contains $
            if re.search('www.americanexpress.com',all_text_page) or re.search('www.citicards.com',all_text_page) or re.search('capitalone.com',all_text_page):
                num_pat = r'(-?\s?\$\d{1,3}(?:,\d{3})*\.\d{2})'
            else:
                num_pat = r'(-?\s?\$?\d{1,3}(?:,\d{3})*\.\d{2})'
            
         # 4 Select proper date pattern for PDF file
            count_p1 = len(re.findall(pattern_1[1:], all_text_page))
            count_p2 = len(re.findall(pattern_2[1:], all_text_page))
            count_p3 = len(re.findall(pattern_3[1:], all_text_page))

            if count_p1 > count_p2 and count_p1 > count_p3:
                kind_extract_pattern = 1
                pat = pattern_1

            elif count_p2 > count_p1 and count_p2 > count_p3:
                kind_extract_pattern = 2
                pat = pattern_2

            elif count_p3 > count_p1 and count_p3 > count_p2:
                kind_extract_pattern = 2
                pat = pattern_3
            else:
                print("ERROR: There is not proper pattern for this file\n\n")
                pdf_dict[base_name]["Status"] = "ERROR: There is not proper pattern for this file"
                return pdf_dict

        #5: Extract Transaction from PDF
            extract_list = []
            for i in range(len(all_text_list)):
                
                line = all_text_list[i]

                if re.search(pat,line) and re.search(num_pat,line):
                    extract_list.append(line.strip())

                elif re.search(pat,line):
                    extract_list.append(line)

                elif re.search(num_pat,line):
                    span = re.search(num_pat,line).span()
                    only_numeric = line[span[0]:span[1]]
                    extract_list.append(only_numeric)

        #6: GET ALL TRANSACTION START WITH DATE OR COMBINE ROW (START WITH DATE) AND ROW NUMBER
            filter_list = []
            # Filter with 53bank
            if re.search('www.53.com/businessbanking',all_text_page):
                for i in range(len(extract_list)):
                    line = extract_list[i]
                    if re.search(pat,line) and re.search(num_pat,line):
                        filter_list.append(line.strip())
            else:
                for i in range(len(extract_list)):
                    line = extract_list[i]
                    # if this line contains both of date and amount => append to filter list
                    if re.search(pat,line) and re.search(num_pat,line):
                        span = re.search(num_pat,line).span()
                        line = line[0:span[1]]
                        filter_list.append(line.strip())

                    # if this line is date = > combine with next line and append to next list
                    elif re.search(pat,line):
                        next_line = extract_list[i+1]

                        # if next line is date get next second line
                        if re.search(pat,next_line) != None:
                            new_line = line +' '+ extract_list[i+2]
                            filter_list.append(new_line)
                        else:
                            new_line = line +' '+ next_line
                            filter_list.append(new_line)

                    # if this line is number: => continue 
                    elif re.search(num_pat,line):
                        continue

        #7: Convert list to dataFrame
            trans_df = pd.DataFrame(filter_list,columns=['Transaction'])

            # Get only transaction which contaims amount
            trans_df = trans_df[~trans_df.Transaction.apply(lambda x: re.search(num_pat,x)).isnull()].reset_index(drop=True)

            # Print total transaction in file pdf
            print(f"File {base_name}.pdf: {trans_df.shape[0]} transactions\n\n")
            pdf_dict[base_name]["Transaction"] = f'File {base_name}.pdf: {trans_df.shape[0]} transactions'
        
            # Create 2 columns Post date and Trans date
            if kind_extract_pattern == 2:
    
                # Define pattern
                kind_2_pat_2d = r'(\d{1,2}[/-]\d{1,2})\s?(\d{1,2}[/-]\d{1,2})'
                kind_2_pat_1d = r'(\d{1,2}[/-]\d{1,2})'
                kind_2_pat_has_year = r'(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'

                # Create filter 
                has_year = trans_df.Transaction.str.contains(pat=kind_2_pat_has_year).sum()/trans_df.shape[0]
                has_2date = trans_df.Transaction.str.contains(pat=kind_2_pat_2d).sum()/trans_df.shape[0]

                # Check case 1: Has year ?
                if has_year > 0.7:
                    trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_has_year)
                    trans_df['Trans_date'] = ['']*trans_df.shape[0]

                # Check case 2: Has two date? Post date and Trans date
                elif has_2date > 0.7:
                    trans_df[['Post_date','Trans_date']] = trans_df.Transaction.str.extract(pat=kind_2_pat_2d)

                    # If there are some transaction having 1 date => get this date to Post day
                    outlier = (trans_df['Transaction'].str.contains(pat=kind_2_pat_2d) == False)
                    trans_df.iloc[outlier,[2]] =trans_df[outlier].Transaction.str.extract(pat=kind_2_pat_1d)

                else:
                    trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_1d)
                    trans_df['Trans_date'] = ['']*trans_df.shape[0] 

            if kind_extract_pattern == 1:
                    # Define pattern
                kind_1_pat_2d = r'([A-Z][a-z]{2}\s?\d{1,2})\s?([A-Z][a-z]{2}\s?\d{1,2})'
                kind_1_pat_1d = r'([A-Z][a-z]{2}\s?\d{1,2})'

                # Create filter 
                has_2date = trans_df.Transaction.str.contains(pat=kind_1_pat_2d).sum()/trans_df.shape[0]

                # Check case: Has two date? Post date and Trans date
                if has_2date > 0.7:
                    trans_df[['Post_date','Trans_date']] = trans_df.Transaction.str.extract(pat=kind_1_pat_2d)

                    # If there are some transaction having 1 date => get this date to Post day
                    outlier = (trans_df['Transaction'].str.contains(pat=kind_1_pat_2d) == False)
                    trans_df.iloc[outlier,[2]] =trans_df[outlier].Transaction.str.extract(pat=kind_1_pat_1d)

                else:
                    trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_1_pat_1d)
                    trans_df['Trans_date'] = ['']*trans_df.shape[0]
            
            # Fill NA
            trans_df = trans_df.fillna('')
     
            # Create Amount columns     
            trans_df['Amount'] = trans_df.Transaction.str.extract(pat=num_pat,expand=False).str.replace(pat=r'[$\s]',repl='',regex=True)

            # Create the Description columns
            trans_df['len_upper']=trans_df[['Trans_date','Post_date']].apply(lambda x: len(x['Trans_date'])+len(x['Post_date']), axis=1)
            trans_df['len_lower'] = trans_df.Transaction.str.extract(num_pat,expand=False).apply(lambda x: len(x))

            trans_df['Description'] = trans_df.apply(lambda x: get_description(trans=x['Transaction'],
                                                                               len_upper=x['len_upper'],
                                                                               len_lower=x['len_lower']),axis=1).str.strip()
            
            # Replace the multiple space to one mutiple space 
            trans_df['Description'] = trans_df.Description.str.replace(regex=True,pat=r'\s+',repl=' ')
                
            # Convert to the excel file
            trans_convert = trans_df[['Transaction', 'Post_date', 'Trans_date','Description', 'Amount']]
            excel_file_path = os.path.join(excel_path, f'{base_name}.xlsx')
            trans_convert.to_excel(excel_writer=excel_file_path,index=False)
            pdf_dict[base_name]["Dataframe"] = trans_convert
            return pdf_dict

### Citi Checking

In [2]:
def citi_bank_to_csv(pdf_path: str,
                     excel_path='C:/Users/trilnd/Desktop/QBO/Test_file/'):
    pattern_1 = r"^([A-Z][a-z]{2}\s?\d{1,2})"
    pattern_2 = r'^((\d{1,2}/\d{1,2})(/\d{2})?)'
    nnum_pat =  r'(-?\s?\$?\d{1,3}(?:,\d{3})*\.\d{2})'
    base_name = os.path.basename(pdf_path)[0:-4]
    pdf_dict = {base_name: {'Status': '',
                            'Transaction': '',
                            'Dataframe': ''}}

    # 1: Check whether the path is correct or wrong
    try:
        with open(pdf_path, 'rb') as file:
            print("Connect file successful")
            pdf_dict[base_name]['Status'] = 'Connect file successful'
    except IOError:
        print("Can not find the file directory or file name does not exist")
        pdf_dict[base_name]['Status'] = 'Can not find the file directory or file name does not exis'
        
    # 2: Implement to convert file to pdf variable and print total number pages
    else:
        try:
            pdf_file = PdfReader(pdf_path, strict=True)
        except Exception as e:
            raise e
        else:
            number_page = pdf_file.getNumPages()

            # 3: Convert pdf file to the string
            all_text_page = ''
            for page_num in range(0, pdf_file.getNumPages()):
                all_text_page += (pdf_file.getPage(page_num).extractText() + '\n')
            all_text_list = [line.strip() for line in all_text_page.split('\n')]

            # : Check if the the file contain text or numeric?
            if re.search(r'.', all_text_page) is None:
                print("This PDF file is Scan data")
                pdf_dict[base_name]['Status'] = "This PDF file is Scan data"
                return None

            # 4 Select proper pattern for PDF file 
            count_p2 = sum([1 for line in all_text_list if re.search(pattern_2, line)])
            if count_p2 > 1:
                pat = pattern_2
            else:
                print("ERROR: There is not proper pattern for this file")
                pdf_dict[base_name]['Status'] = "ERROR: There is not proper pattern for this file"
                return None

            # 5: Extract Transaction from PDF
            extract_list = []
            for i in range(len(all_text_list)):
                line = all_text_list[i]
                if re.search(pat, line):
                    list_trans = [line for line in all_text_list[i:(i + 6)] if line != '']
                    extract_list.append(list_trans)

                # Check whether last value is date value? if yes, replace None
            for line in extract_list:
                line[-1] = re.sub(pattern=pat, repl='', string=line[-1])

            # 6: Convert trans to Data frame
            trans_df = pd.DataFrame(extract_list,
                                    columns=['Date', 'Description', 'Amount', 'Balance', 'Sub_description_1',
                                             'Sub_description_2'])
            # Print total transaction in DF
            print(f"File {base_name}.pdf: {trans_df.shape[0]} transactions\n\n")
            pdf_dict[base_name]['Transaction'] = f"File {base_name}.pdf: {trans_df.shape[0]} transactions"

            trans_df['Description'] = trans_df.Description.str.replace(regex=True, pat=r'\s+', repl=' ')
            trans_df['Sub_description_1'] = trans_df.Sub_description_1.str.replace(regex=True, pat=r'\s+', repl=' ')
            trans_df['Sub_description_2'] = trans_df.Sub_description_2.str.replace(regex=True, pat=r'\s+', repl=' ')

            trans_convert = trans_df[['Date', 'Description', 'Sub_description_1', 'Sub_description_2', 'Amount']]

            # 7: Convert to excel file
            trans_convert.to_excel(excel_writer=f'{excel_path}{base_name}.xlsx', index=False)
            pdf_dict[base_name]['Dataframe'] = trans_convert
            return pdf_dict


### Bank Of Hope/ Bank Panacea

In [3]:
def hope_bank_to_csv(pdf_path: str,
                     excel_path='C:/Users/trilnd/Desktop/QBO/Test_file/'):
       #Function to get description

    def get_description(trans,len_upper,len_lower):
        # check space: 
        check_space = pat+'(?=\s)'
        if re.search(check_space,trans) != None:
            return trans[len_upper+1:(len(trans)-len_lower)]
        else:
            return trans[(len_upper):(len(trans)-len_lower)]

    pattern_1 = r"^([A-Z][a-z]{2}\s?\d{1,2})"
    pattern_2 = r'^((\d{1,2}/\d{1,2})(/\d{4})?)'
    num_pat = r'(-?\s?\$\d{1,3}(?:,\d{3})*\.\d{2})'
    base_name = os.path.basename(pdf_path)[0:-4]
    pdf_dict = {base_name:{'Status':'',
                           'Transaction':'',
                           'Dataframe':''}}
    
    #1: Check whether the path is correct or wrong
    try:
        with open(pdf_path,'rb') as file:
            print(f"{base_name}: Connect file successful")
            pdf_dict[base_name]["Status"] = "Connect file successful"
            
    except IOError:
            print("Can not find the file directory or file name does not exist")
            pdf_dict[base_name]["Status"] = "Can not find the file directory or file name does not exist"
            return pdf_dict

    #2: Implement to convert file to pdf variable and print total number pages
    else:
        try:
            pdf_file = fitz.open(pdf_path)
        except Exception as e:
            raise e
        else:
            
#################################### GET TRANSACTION ########################################### 

        #3: Convert pdf file to the string 
            all_text_page =''
            for page in pdf_file:
                text = page.get_text('text')
                all_text_page+= (text+'\n')

            if re.search(r'.',all_text_page) == None:
                raise Exception("This file without text data")
            else:
                all_text_list = [line.strip() for line in all_text_page.split('\n')]
            
        # : Check if the the file contain text or numeric?
            if re.search(r'.',all_text_page) is None:
                print("This PDF file is Scan data")
                pdf_dict[base_name]["Status"] = "This PDF file is Scan data"
                return pdf_dict
                     
        #4 Select proper date pattern for PDF file
            count_p1 = len(re.findall(pattern_1[1:],all_text_page))
            count_p2 = len(re.findall(pattern_2[1:],all_text_page))
            
            if count_p1 > count_p2:
                kind_extract_pattern = 1
                pat = pattern_1
                #print("Chose the patern 1")
                  
            elif  count_p1 < count_p2:
                kind_extract_pattern = 2
                pat = pattern_2
                #print("Chose the patern 2")
            else:
                print("ERROR: There is not proper pattern for this file\n\n")
                pdf_dict[base_name]["Status"] = "ERROR: There is not proper pattern for this file"
                return pdf_dict

        #5: Extract Transaction from PDF
            extract_list = []
            
            if re.search('capitalone.com', all_text_page):
                for i, line in enumerate(all_text_list):
                    if re.search(pat, line) and re.search(num_pat, line):
                        extract_list.append(line.strip())

                    elif re.search(pat, line) and re.search(num_pat, all_text_list[i + 3]):
                        trans = all_text_list[i:i + 4]
                        extract_list.append(' '.join(trans))
            else:
                for i, line in enumerate(all_text_list):
                    if re.search(pat,line) and re.search(num_pat,line):
                        extract_list.append(line.strip())

                    elif re.search(pat,line):
                        extract_list.append(line+' '+all_text_list[i+1])

                    elif re.search(num_pat,line):
                        span = re.search(num_pat,line).span()
                        only_numeric = line[span[0]:span[1]]
                        extract_list.append(only_numeric)

        #6: GET ALL TRANSACTION START WITH DATE OR COMBINE ROW (START WITH DATE) AND ROW NUMBER
            filter_list = []
            if re.search('bankofthewest.com', all_text_page) or re.search('capitalone.com', all_text_page):
                for i, line in enumerate(extract_list):
                    if re.search(pat, line) and re.search(num_pat, line):
                        filter_list.append(line.strip())

            else:
                for i, line in enumerate(extract_list):
                    if len(extract_list)-1 > i:
                        # if this line contains both of date and amount => append to filter list
                        if re.search(pat,line) and re.search(num_pat,line):
                            span = re.search(num_pat,line).span()
                            line = line[0:span[1]]
                            filter_list.append(line.strip())

                        # if this line is date = > combine with next line and append to next list
                        elif re.search(pat,line):
                            next_line = extract_list[i+1]

                            # if next line is date get next second line
                            if re.search(pat,next_line) != None:
                                try:
                                    new_line = line +' '+ extract_list[i+2]
                                    filter_list.append(new_line)
                                except IndexError:
                                    pass
                            else:
                                new_line = line +' '+ next_line
                                filter_list.append(new_line)

                        # if this line is number: => continue 
                        elif re.search(num_pat,line):
                            continue
                        
#################################### GET CHECK NUMBER ###########################################

        # Pattern define:
            pat_check_no = r'\b\d+\*?\b'
            pat_date =  r'((\d{1,2}/\d{1,2})(/\d{4})?)'
            pat_num = r'(-?\s?\$\d{1,3}(?:,\d{3})*\.\d{2})'
            
         #1: Get Page and Coodinator contain "Checks" word
            String = 'Checks'
            Pagelist = []
            for i in range(0, number_page):
                PageObj = pdf_file[i]
                Text = PageObj.get_text('text')
                ReSearch = re.search(String, Text, flags=re.IGNORECASE)
                if ReSearch != None:
                    # print(re.findall(String,Text))
                    info = (i, ReSearch.span()[0])
                    Pagelist.append(info)
                    
        #2: Convert string to list
            checkno_text_page = ''
            for i in Pagelist:
                checkno_text_page += (pdf_file[i[0]].get_text('text')[i[1]:] + '\n')
                
        # 3: Get line without Alphabelt digit
            filter_checkno_list = [line.strip() for line in checkno_text_page.split('\n') if re.search(r'[a-zA-Z+]',line) is None]
        
        # 4: Get only checkno line
            check_no_list = []
            for i in range(len(filter_checkno_list)):
                if i <len(filter_checkno_list)-1:
                    line = filter_checkno_list[i]
                    next_line = filter_checkno_list[i+1]
                    if re.search(pat_check_no,line) and re.search(pat_date,line):
                        if re.search(pat_num,next_line):
                            check_no_line = line+' '+next_line
                            check_no_list.append(check_no_line.split())
            

################################### CONVERT TO DATAFRAME AND EXPORT TO FILE CSV ###############################################

            #################### TRANSACTION DATAFRAME #########################
            trans_df = pd.DataFrame(filter_list,columns=['Transaction'])

            # Get only transaction which contaims amount
            trans_df = trans_df[~trans_df.Transaction.apply(lambda x: re.search(num_pat,x)).isnull()].reset_index(drop=True)

            # Print total transaction in file pdf
            print(f"File {base_name}.pdf: {trans_df.shape[0]} transactions\n\n")
            pdf_dict[base_name]["Transaction"] = f'File {base_name}.pdf: {trans_df.shape[0]} transactions'
        
            # Create 2 columns Post date and Trans date
            if kind_extract_pattern == 2:
    
                # Define pattern
                kind_2_pat_2d = r'(\d{1,2}[/-]\d{1,2})\s?(\d{1,2}[/-]\d{1,2})'
                kind_2_pat_1d = r'(\d{1,2}[/-]\d{1,2})'
                kind_2_pat_has_year = r'(\d{1,2}[/-]\d{1,2}[/-]\d{4})'

                # Create filter 
                has_year = trans_df.Transaction.str.contains(pat=kind_2_pat_has_year).sum()/trans_df.shape[0]
                has_2date = trans_df.Transaction.str.contains(pat=kind_2_pat_2d).sum()/trans_df.shape[0]

                # Check case 1: Has year ?
                if has_year > 0.7:
                    trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_has_year)
                    trans_df['Trans_date'] = ['']*trans_df.shape[0]

                # Check case 2: Has two date? Post date and Trans date
                elif has_2date > 0.7:
                    trans_df[['Post_date','Trans_date']] = trans_df.Transaction.str.extract(pat=kind_2_pat_2d)

                    # If there are some transaction having 1 date => get this date to Post day
                    outlier = (trans_df['Transaction'].str.contains(pat=kind_2_pat_2d) == False)
                    trans_df.iloc[outlier,[2]] =trans_df[outlier].Transaction.str.extract(pat=kind_2_pat_1d)

                else:
                    trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_1d)
                    trans_df['Trans_date'] = ['']*trans_df.shape[0] 
                    
            elif kind_extract_pattern == 1:
                 # Define pattern
                kind_1_pat_2d = r'([A-Z][a-z]{2}\s?\d{1,2})\s?([A-Z][a-z]{2}\s?\d{1,2})'
                kind_1_pat_1d = r'([A-Z][a-z]{2}\s?\d{1,2})'
                kind_1_pat_has_year = r'([A-Z][a-z]{2}\s?\d{1,2},?\s?\d{2,4})'

                # Create filter
                has_2date = trans_df.Transaction.str.contains(pat=kind_1_pat_2d).sum() / trans_df.shape[0]
                has_year = trans_df.Transaction.str.contains(pat=kind_1_pat_has_year).sum() / trans_df.shape[0]

                # Check case 1: Has year ?
                if has_year > 0.7:
                    trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_1_pat_has_year)
                    trans_df['Trans_date'] = [''] * trans_df.shape[0]

                # Check case: Has two date? Post date and Trans date
                elif has_2date > 0.7:
                    trans_df[['Post_date', 'Trans_date']] = trans_df.Transaction.str.extract(pat=kind_1_pat_2d)

                    # If there are some transaction having 1 date => get this date to Post day
                    outlier = (trans_df['Transaction'].str.contains(pat=kind_1_pat_2d) == False)
                    trans_df.iloc[outlier, [2]] = trans_df[outlier].Transaction.str.extract(pat=kind_1_pat_1d)

                else:
                    trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_1_pat_1d)
                    trans_df['Trans_date'] = [''] * trans_df.shape[0]
                

            # Fill NA
            trans_df = trans_df.fillna('')
     
            # Create Amount columns     
            trans_df['Amount'] = trans_df.Transaction.str.extract(pat=num_pat,expand=False).str.replace(pat=r'[$\s]',repl='',regex=True)

            # Create the Description columns
            trans_df['len_upper']=trans_df[['Trans_date','Post_date']].apply(lambda x: len(x['Trans_date'])+len(x['Post_date']), axis=1)
            trans_df['len_lower'] = trans_df.Transaction.str.extract(num_pat,expand=False).apply(lambda x: len(x))

            trans_df['Description'] = trans_df.apply(lambda x: get_description(trans=x['Transaction'],
                                                                               len_upper=x['len_upper'],
                                                                               len_lower=x['len_lower']),axis=1).str.strip()
            
            # Replace the multiple space to one mutiple space 
            trans_df['Description'] = trans_df.Description.str.replace(regex=True,pat=r'\s+',repl=' ')
            columns = ['Transaction', 'Trans_date', 'Post_date', 'Amount','Description']
            
            # Convert to the excel file
            excel_file_path = os.path.join(excel_path, f'{base_name}.xlsx')
            trans_df[columns].to_excel(excel_writer=excel_file_path,index=False)  
            pdf_dict[base_name]["Dataframe"] = trans_df[columns]
            
            ####################### CHECK NO DATAFRAME #####################################
            check_no_df = pd.DataFrame(check_no_list,columns=['Check_no','Date','Amount'])
            check_no_df['Amount'] = check_no_df['Amount'].str.replace('$','',regex=False)
            # 6: Update file excel created before with new sheet "Check No"
            
            # if check path is valid
            if os.path.exists(excel_file_path):
                with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                    check_no_df.to_excel(writer, sheet_name='Check_no', index=False)
            else:
                check_no_df.to_excel(path, sheet_name='Check_no', index=False)
                
            pdf_dict[base_name]["DF_Check_No"] = check_no_df
            
            return pdf_dict

## Function to get Check No

In [4]:
# Kind 3: Function to get Check No in Bank Statement
def get_check_no(pdf_path: str,
                 excel_path: str = 'C:/Users/trilnd/Desktop/QBO/Test_file/'):
    pattern_1 = r"^([A-Z][a-z]{2}\s?\d{1,2})"
    pattern_2 = r'^((\d{1,2}/\d{1,2})(/\d{2})?)'
    num_pat = r'(-?\s?\$?\d+,?\d*\.\d*)'
    pattern_check_no = r'\b\d{4,5}\b'
    base_name = os.path.basename(pdf_path)[:-4]
    pdf_dict = {base_name: {'Status': '',
                            'Transaction': '',
                            'Dataframe': ''}}

    try:
        with open(pdf_path, 'rb') as file:
            print("Connect file successful")
    except IOError:
        print("Can not find the file directory or file name does not exist")
        pdf_dict['Status'] = "Can not find the file directory or file name does not exist"

    # 2: Implement to convert file to pdf variable and print total number pages
    else:
        try:
            pdf_file = PdfReader(pdf_path, strict=True)
        except Exception as e:
            raise e
        else:
            number_page = pdf_file.getNumPages()

            # Get Page and Coodinator contain "Checks" word
            String = 'Checks'
            Pagelist = []
            for i in range(0, number_page):
                PageObj = pdf_file.getPage(i)
                Text = PageObj.extractText()
                ReSearch = re.search(String, Text, flags=re.IGNORECASE)
                if ReSearch != None:
                    # print(re.findall(String,Text))
                    info = (i, ReSearch.span()[0])
                    Pagelist.append(info)

            # 3: Convert pdf file to the string
            all_text_page = ''
            for i in Pagelist:
                all_text_page += (pdf_file.getPage(i[0]).extract_text()[i[1]:] + '\n')

            all_text_list = [line.strip() for line in all_text_page.split('\n')]

            # 4 Select proper pattern for PDF file
            list_check_no = [line for line in all_text_list if
                             re.search(pattern=pattern_check_no, string=line) and re.search(num_pat, line)]

            # 5: Create DF
            df = pd.DataFrame({"Check no": list_check_no})
            print(f"File {base_name}.pdf: {df.shape[0]} Checks")
            pdf_dict[base_name]['Transaction'] = f"File {base_name}.pdf: {df.shape[0]} Checks"

            # 6: Update file excel created before with new sheet "Check No"
            path = os.path.join(excel_path, f'{base_name}.xlsx')

            # if check path is valid
            if os.path.exists(path):
                with pd.ExcelWriter(path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                    df.to_excel(writer, sheet_name='Check_no', index=False)
            else:
                df.to_excel(path, sheet_name='Check_no', index=False)
                pdf_dict[base_name]['Dataframe'] = df
            return pdf_dict

## Fucntion to convert folder containing PDF files to excel file

In [5]:
def convert_pdffolder_to_excelfolder(pdf_path: str, xlsx_path: str, function_kind: int):
    function_list = [convert_pdf_to_csv,
                     citi_bank_to_csv,
                     get_check_no,
                     hope_bank_to_csv]
    function = function_list[function_kind - 1]
    list_pdfs = []
    list_file = [os.path.join(pdf_path, f) for f in os.listdir(pdf_path) if f.endswith('.pdf')]
    # kind 1: Popular type: convert PDF to excel (except Citi Checking)
    if len(list_file) == 0:
        print("There is no file PDF. Please choose again")
        # return "There is no file PDF. Please choose again"
    else:
        if function != get_check_no:
            list_file = [file for file in list_file if
                         os.path.exists(os.path.join(xlsx_path, os.path.basename(file)[:-4] + '.xlsx')) is False]
            for file in list_file:
                try:
                    dict_pdf = function(pdf_path=file, excel_path=xlsx_path)
                except Exception:
                    continue
                else:
                    list_pdfs.append(dict_pdf)
        else:
            for file in list_file:
                try:
                    dict_pdf = get_check_no(pdf_path=file, excel_path=xlsx_path)
                except Exception:
                    continue
                else:
                    list_pdfs.append(dict_pdf)

        # Create message to display in GUI
        status_pdf = [info['Status'] for dict_pdf in list_pdfs for (key, info) in dict_pdf.items()]
        trans_pdf = [info['Transaction'] for dict_pdf in list_pdfs for (key, info) in dict_pdf.items()]
        filename_pdf = [key for dict_pdf in list_pdfs for (key, info) in dict_pdf.items()]
        message = '\n'.join([f'{n}.pdf\n{s}\n{t}\n' for n, s, t in zip(filename_pdf, status_pdf, trans_pdf)])

        # Get total Transaction in dataframe
        valid_dfs = [info['Dataframe']
                     for dict_pdf in list_pdfs
                     for (key, info) in dict_pdf.items()
                     if isinstance(info['Dataframe'], pd.DataFrame)]

        # sum transaction of total PDF file
        total = sum([df.shape[0] for df in valid_dfs])
        summary = f"In {len(list_file)} files,There are {len(valid_dfs)} valid files with Total {total} transactions"
        
        return list_pdfs
        #return f'{message}\n{summary}'

# Test Credit file

## TEST FILE CITI PDF

In [12]:
citi_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Citi CC.pdf'
citi_dict = convert_pdf_to_csv(citi_dir)

Citi CC: Connect file successful
File Citi CC.pdf: 50 transactions




In [13]:
citi_dict['Citi CC']['Dataframe'].head()

,Transaction,Trans_date,Post_date,Amount,Description
0,08/24/22-09/23/22 $84.00,4/22,08/2,84.00,-09/23/22
1,"08/26 PAYMENT THANK YOU -$15,581.45",08/26,,"-15,581.45",PAYMENT THANK YOU
2,"09/12 PAYMENT THANK YOU -$8,905.46",09/12,,"-8,905.46",PAYMENT THANK YOU
3,"09/13 ONLINE PAYMENT, THANK YOU -$1,313.60",09/13,,"-1,313.60","ONLINE PAYMENT, THANK YOU"
4,"09/13 ONLINE PAYMENT, THANK YOU -$1,368.59",09/13,,"-1,368.59","ONLINE PAYMENT, THANK YOU"


## TEST FILE CREDIT

In [14]:
credit_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/credit_card.pdf'
credit_dict = convert_pdf_to_csv(credit_dir)

credit_card: Connect file successful
File credit_card.pdf: 237 transactions




In [15]:
credit_dict['credit_card']['Dataframe'].head()

,Transaction,Trans_date,Post_date,Amount,Description
0,"Sep 28, 2022 For online and phone payments, the $1,150.83",Sep 28,,"1,150.83",", 2022 For online and phone payments, the"
1,"Aug 04, 2022 - Sep 03, 2022 | 31 days in Billing Cycle $52.08",Aug 04,,52.08,", 2022 - Sep 03, 2022 | 31 days in Billing Cycle"
2,"Aug 04, 2022 - Sep 03, 2022 | 31 days in Billing Cycle Aug 5 Aug 5 CAPITAL ONE MOBILE PYMTAut...",Aug 5,Aug 5,-807.84,"22 - Sep 03, 2022 | 31 days in Billing Cycle Aug 5 Aug 5 CAPITAL ONE MOBILE PYMTAuthDate 05-Aug"
3,Aug 4 Aug 4 CAPITAL ONE MOBILE PYMTAuthDate 04-Aug - $874.68,Aug 4,Aug 4,-874.68,CAPITAL ONE MOBILE PYMTAuthDate 04-Aug
4,Aug 5 Aug 5 CAPITAL ONE MOBILE PYMTAuthDate 05-Aug - $807.84,Aug 5,Aug 5,-807.84,CAPITAL ONE MOBILE PYMTAuthDate 05-Aug


## Check BofA file

In [16]:
BofA_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/BofA CC.pdf'
BofA_dict = convert_pdf_to_csv(BofA_dir)

BofA CC: Connect file successful
File BofA CC.pdf: 51 transactions




In [18]:
BofA_dict['BofA CC']['Dataframe'].sample(10,random_state=26)

,Transaction,Trans_date,Post_date,Amount,Description
5,06/07 06/05 SUSHI MASA SUGAR LAND TX 25247802157000357072305 437.78,06/05,06/07,437.78,SUSHI MASA SUGAR LAND TX 25247802157000357072305
49,07/04 07/02 BENCO DENTAL CO 7178257781 PA 55548072184081228415386 666.62,07/02,07/04,666.62,BENCO DENTAL CO 7178257781 PA 55548072184081228415386
20,06/13 06/11 eBay O*10-08735-82863 San Jose CA 15270212162000050010137 118.14,06/11,06/13,118.14,eBay O*10-08735-82863 San Jose CA 15270212162000050010137
2,06/27 06/24 AMZN MKTP US AMZN.COM/ AMZN.COM/BILLWA 55310202175083001846381 - 38.24,06/24,06/27,-38.24,AMZN MKTP US AMZN.COM/ AMZN.COM/BILLWA 55310202175083001846381
14,06/24 06/23 REI #55 HOUSTON HOUSTON TX 55432862175200798065702 - 102.84,06/23,06/24,-102.84,REI #55 HOUSTON HOUSTON TX 55432862175200798065702
10,06/24 06/22 THE HOME DEPOT #6530 ROSENBERG TX 52707152174010198551249 6.80,06/22,06/24,6.80,THE HOME DEPOT #6530 ROSENBERG TX 52707152174010198551249
47,07/04 07/02 bath&bodyworks.com Reynoldsburg OH 12302022183000740690139 1.74,07/02,07/04,1.74,bath&bodyworks.com Reynoldsburg OH 12302022183000740690139
9,06/24 06/22 THE HOME DEPOT #6530 ROSENBERG TX 52707152174010198553484 27.94,06/22,06/24,27.94,THE HOME DEPOT #6530 ROSENBERG TX 52707152174010198553484
11,06/27 06/25 AMZN MKTP US*115AT5ZK3 AMZN.COM/BILLWA 55310202176083749769308 101.98,06/25,06/27,101.98,AMZN MKTP US*115AT5ZK3 AMZN.COM/BILLWA 55310202176083749769308
42,06/29 06/28 UNIVERSAL TELCOM 504-9623500 LA 75306372179172600480832 148.93,06/28,06/29,148.93,UNIVERSAL TELCOM 504-9623500 LA 75306372179172600480832


## TEST US BANK CC

In [19]:
US_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/US Bank CC.pdf'
US_dict = convert_pdf_to_csv(US_dir)

US Bank CC: Connect file successful
File US Bank CC.pdf: 57 transactions




In [20]:
US_dict['US Bank CC'][ 'Transaction']

'File US Bank CC.pdf: 57 transactions'

In [22]:
US_dict['US Bank CC']['Dataframe'].sample(10,random_state=26)

,Transaction,Trans_date,Post_date,Amount,Description
10,08/2208/195372SEAMLSSCAFECORLEONE SEAMLESS.COM NY $62.62,08/19,08/22,62.62,5372SEAMLSSCAFECORLEONE SEAMLESS.COM NY
9,08/2208/218653GOOGLE *YouTubePremium g.co/helppay# CA $11.99,08/21,08/22,11.99,8653GOOGLE *YouTubePremium g.co/helppay# CA
5,09/0609/05040776 - STANTON 76 STANTON CA $76.40,09/05,09/06,76.40,040776 - STANTON 76 STANTON CA
36,09/0609/030947SHABULIN ARCADIA CA $734.62,09/03,09/06,734.62,0947SHABULIN ARCADIA CA
11,08/2208/190371HOT SALSA PARAMOUNT CA $75.95,08/19,08/22,75.95,0371HOT SALSA PARAMOUNT CA
56,08/19/2022 -09/20/2022 $0.00,9/20,08/1,0.00,2 -09/20/2022
2,08/19/2022 -09/20/2022 $78.04,9/20,08/1,78.04,2 -09/20/2022
22,09/2009/192097PAYPAL *LYFT RIDE SUN 402-935-7733 CA $44.99,09/19,09/20,44.99,2097PAYPAL *LYFT RIDE SUN 402-935-7733 CA
37,09/0609/046318FEDEX 67444631 800-4633339 TN $23.53,09/04,09/06,23.53,6318FEDEX 67444631 800-4633339 TN
39,09/0609/025652PB LEASING 844-256-6444 CT $192.99,09/02,09/06,192.99,5652PB LEASING 844-256-6444 CT


## Test Fifth Third Bank

In [24]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.04.pdf' 
fifththird_dir = convert_pdf_to_csv(_dir)

2022.04: Connect file successful
File 2022.04.pdf: 103 transactions




In [25]:
fifththird_dir['2022.04']['Dataframe']

,Transaction,Trans_date,Post_date,Amount,Description
0,"04/01 Beginning Balance $113,419.18 Number of Days in Period 30",,04/01,"113,419.18","lance $113,419.18 Number of Days in Period 30"
1,"04/30 Ending Balance $154,725.70",,04/30,"154,725.70","ce $154,725.70"
2,04/04 76.13 MERCH BANKCARD BILLNG 520004511336 ALEXANDER CHOE DDS INC MERCH BANKCARD 520004511336,,04/04,76.13,MERCH BANKCARD BILLNG 520004511336 ALEXANDER CHOE DDS INC MERCH BANKCARD 520004511336
3,04/05 20.52 PAY PLUS PPSACCOUNT ACHTRANS 452579291 ZP Account 5 040522,,04/05,20.52,PAY PLUS PPSACCOUNT ACHTRANS 452579291 ZP Account 5 040522
4,"04/05 1,411.22 5/3 COMMRCL LN #XXXXXXXXXXX0018 PAID BY AUTO BILLPAYER",,04/05,"1,411.22",5/3 COMMRCL LN #XXXXXXXXXXX0018 PAID BY AUTO BILLPAYER
...,...,...,...,...,...
98,"04/05 118,686.03 04/14 138,164.65 04/25 149,808.67",,04/05,"118,686.03","04/14 138,164.65 04/25 149,808.67"
99,"04/06 121,379.38 04/15 138,328.85 04/26 154,327.62",,04/06,"121,379.38","04/15 138,328.85 04/26 154,327.62"
100,"04/07 131,075.52 04/18 142,718.05 04/27 159,769.53",,04/07,"131,075.52","04/18 142,718.05 04/27 159,769.53"
101,"04/08 129,935.60 04/19 145,512.83 04/28 154,469.98",,04/08,"129,935.60","04/19 145,512.83 04/28 154,469.98"


## TEST AMEX CC

- This file is in scope L2: a transaction is multiple line

In [27]:
amex_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/AMEX CC.pdf'
amex_dict = convert_pdf_to_csv(amex_dir)

AMEX CC: Connect file successful
File AMEX CC.pdf: 292 transactions




In [29]:
amex_dict['AMEX CC']['Dataframe'].sample(10,random_state=26)

,Transaction,Trans_date,Post_date,Amount,Description
195,08/11/22 AMAZON TIPS*4I6XA3LK3 AMZN.COM/BILL WA $5.00,,08/11/22,5.00,AMAZON TIPS*4I6XA3LK3 AMZN.COM/BILL WA
5,08/21/22 JAMES VARTANIAN AMAZON.COM -$22.36,,08/21/22,-22.36,JAMES VARTANIAN AMAZON.COM
197,08/11/22 TARGET.COM 800-591-3869 MN $32.30,,08/11/22,32.30,TARGET.COM 800-591-3869 MN
146,08/06/22 AMZN MKTP US*8L1JL3PE3 AMZN.COM/BILL WA $7.53,,08/06/22,7.53,AMZN MKTP US*8L1JL3PE3 AMZN.COM/BILL WA
72,07/28/22 AMZN MKTP US*QT85R3XM3 AMZN.COM/BILL WA $38.78,,07/28/22,38.78,AMZN MKTP US*QT85R3XM3 AMZN.COM/BILL WA
231,08/16/22 PAYPAL *UBER EATS 8665761039 CA $2.00,,08/16/22,2.00,PAYPAL *UBER EATS 8665761039 CA
159,08/08/22 SIRIUS XMRADIO INC. 888-635-5144 NY $8.69,,08/08/22,8.69,SIRIUS XMRADIO INC. 888-635-5144 NY
117,08/02/22 TJMAXX ALISO VIEJO CA $149.17,,08/02/22,149.17,TJMAXX ALISO VIEJO CA
136,08/05/22 ACRYSTAL COVE NEWPORT COAST CA $141.20,,08/05/22,141.20,ACRYSTAL COVE NEWPORT COAST CA
92,07/30/22 AMAZON TIPS*9L65G7FV3 AMZN.COM/BILL WA $4.00,,07/30/22,4.00,AMAZON TIPS*9L65G7FV3 AMZN.COM/BILL WA


## Test file has 2 year: American Express

In [30]:
american_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/American_2y.pdf'
ame_express_2y_dict = convert_pdf_to_csv(american_dir)

American_2y: Connect file successful
File American_2y.pdf: 179 transactions




In [32]:
ame_express_2y_dict['American_2y']['Dataframe'].sample(10,random_state=26)

,Transaction,Trans_date,Post_date,Amount,Description
125,01/06/22 BT*DD *DOORDASH BCDTOFUHO SAN FRANCISCO CA $72.53,,01/06/22,72.53,BT*DD *DOORDASH BCDTOFUHO SAN FRANCISCO CA
166,01/14/22 DOORDASH*URTH CAFFE SAN FRANCISCO CA $17.81,,01/14/22,17.81,DOORDASH*URTH CAFFE SAN FRANCISCO CA
71,12/21/21 OFFICE DEPOT #5125 000005125 SIGNAL HILL CA $50.21,,12/21/21,50.21,OFFICE DEPOT #5125 000005125 SIGNAL HILL CA
30,12/29/21 STATE FARM INSURANCE BLOOMINGTON IL $779.79,,12/29/21,779.79,STATE FARM INSURANCE BLOOMINGTON IL
123,01/06/22 MYGLENDALE* GLENDALEFL GLENDALE CA $120.23,,01/06/22,120.23,MYGLENDALE* GLENDALEFL GLENDALE CA
155,01/12/22 BT*DD *DOORDASH ROJIBAKER SAN FRANCISCO CA $37.21,,01/12/22,37.21,BT*DD *DOORDASH ROJIBAKER SAN FRANCISCO CA
126,01/06/22 BT*DD *DOORDASH WALTERSCA SAN FRANCISCO CA $17.03,,01/06/22,17.03,BT*DD *DOORDASH WALTERSCA SAN FRANCISCO CA
42,01/01/22 NEXTIVA*VOIP SERVICE 800-983-4289 AZ $162.42,,01/01/22,162.42,NEXTIVA*VOIP SERVICE 800-983-4289 AZ
108,01/03/22 BT*DD *DOORDASH NONGLA SAN FRANCISCO CA $55.76,,01/03/22,55.76,BT*DD *DOORDASH NONGLA SAN FRANCISCO CA
85,12/27/21 INSTACART SAN FRANCISCO CA $86.23,,12/27/21,86.23,INSTACART SAN FRANCISCO CA


# Test Bank Statement

## BOAF

### Get deposit

In [33]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/BofA.pdf'
BS_bofa_dict = convert_pdf_to_csv(_dir)

BofA: Connect file successful
File BofA.pdf: 54 transactions




In [35]:
BS_bofa_dict['BofA']['Dataframe'].sample(10,random_state=26)

,Transaction,Trans_date,Post_date,Amount,Description
9,"08/10/22 FISERV MERCHANT DES:DEPOSIT ID:372731789882 INDN:SEAN H YU DDS INC CO 3,272.04",,08/10/22,"3,272.04",FISERV MERCHANT DES:DEPOSIT ID:372731789882 INDN:SEAN H YU DDS INC CO
50,"08/05 108,212.49",,,"108,212.49",8/05
5,08/05/22 FISERV MERCHANT DES:DEPOSIT ID:372731789882 INDN:SEAN H YU DDS INC CO 951.10,,08/05/22,951.10,FISERV MERCHANT DES:DEPOSIT ID:372731789882 INDN:SEAN H YU DDS INC CO
36,08/18/22 PAYROLL SERVICE DES:9F6N ID:9F6N 9F6N INDN:SEAN H YU DDS INC CO -117.83,,08/18/22,-117.83,PAYROLL SERVICE DES:9F6N ID:9F6N 9F6N INDN:SEAN H YU DDS INC CO
2,"08/02/22 FISERV MERCHANT DES:DEPOSIT ID:372731789882 INDN:SEAN H YU DDS INC CO 2,048.00",,08/02/22,"2,048.00",FISERV MERCHANT DES:DEPOSIT ID:372731789882 INDN:SEAN H YU DDS INC CO
10,"08/11/22 FISERV MERCHANT DES:DEPOSIT ID:372731789882 INDN:SEAN H YU DDS INC CO 3,932.05",,08/11/22,"3,932.05",FISERV MERCHANT DES:DEPOSIT ID:372731789882 INDN:SEAN H YU DDS INC CO
20,"08/22/22 FISERV MERCHANT DES:DEPOSIT ID:372731789882 INDN:SEAN H YU DDS INC CO 2,932.85",,08/22/22,"2,932.85",FISERV MERCHANT DES:DEPOSIT ID:372731789882 INDN:SEAN H YU DDS INC CO
39,08/25/22 ORANGE COUNTY DES:TAX COLL. ID:0727423 INDN:Sean h yu dds inc CO ID:99560009...,,08/25/22,"-1,043.79",ORANGE COUNTY DES:TAX COLL. ID:0727423 INDN:Sean h yu dds inc CO ID:9956000928
37,"08/22/22 Online Banking payment to CRD 2867 Confirmation# 0111276183 -9,100.17",,08/22/22,"-9,100.17",Online Banking payment to CRD 2867 Confirmation# 0111276183
47,"08/02 98,646.13",,,"98,646.13",8/02


### Get Check No

In [36]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/BofA.pdf'
BS_bofa_checkno = get_check_no(_dir)

Connect file successful
File BofA.pdf: 6 Checks


In [37]:
BS_bofa_checkno

{'BofA': {'Status': '',
  'Transaction': 'File BofA.pdf: 6 Checks',
  'Dataframe': ''}}

## US Bank CHK

### Deposit and Debts

In [38]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/US Bank CHK.pdf'
BS_USCHK_df = convert_pdf_to_csv(_dir)

US Bank CHK: Connect file successful
File US Bank CHK.pdf: 82 transactions




In [28]:
BS_USCHK_df['US Bank CHK']['DataFrame'].sample(10,random_state=26)

,Transaction,Trans_date,Post_date,Amount,Description
4,"Sep2MERCH 8032426499 BUENA PA DEPOSIT 7,887.83",,Sep2,"7,887.83",MERCH 8032426499 BUENA PA DEPOSIT
60,"Sep2 103,981.94",,Sep2,"103,981.94",
70,"Sep20 114,769.40",,Sep20,"114,769.40",
71,"Sep21 112,762.45",,Sep21,"112,762.45",
73,"Sep26 201,901.98",,Sep26,"201,901.98",
2,"Sep 30, 2022 Sep1MERCH 8032426499 BUENA PA DEPOSIT $ 8,970.86",,Sep 30,"8,970.86",", 2022 Sep1MERCH 8032426499 BUENA PA DEPOSIT $"
31,"Sep26MERCH 8032426499 BUENA PA DEPOSIT 9,759.99",,Sep26,"9,759.99",MERCH 8032426499 BUENA PA DEPOSIT
20,"Sep19MERCH 8032426499 BUENA PA DEPOSIT 4,013.53",,Sep19,"4,013.53",MERCH 8032426499 BUENA PA DEPOSIT
33,"Sep27MERCH 8032426499 BUENA PA DEPOSIT 3,927.07",,Sep27,"3,927.07",MERCH 8032426499 BUENA PA DEPOSIT
41,"Sep2Internet Banking Payment To Credit Card *************4710 3,464.56",,Sep2,"3,464.56",Internet Banking Payment To Credit Card *************4710


### Get Check No

In [29]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/US Bank CHK.pdf'

In [30]:
BS_USCHK_checkno = get_check_no(_dir)

Connect file successful
File US Bank CHK.pdf: 18 Checks


In [31]:
BS_USCHK_checkno

{'US Bank CHK': {'Status': '',
  'Transaction': 'File US Bank CHK.pdf: 18 Checks',
  'Dataframe': ''}}

## Union Bank

### USE TABULA TO EXTRACT THE TABLE

In [32]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/Union Bank.pdf'
#BS_Union_df = convert_pdf_to_csv(_dir,2,'Union_bank')

In [33]:
dfs = tabula.read_pdf(input_path=_dir,pages='all',multiple_tables=True,stream=True)

In [34]:
len(dfs)

4

In [35]:
union_bank_df = pd.concat(dfs,ignore_index=False)
union_bank_df.drop(columns='Unnamed: 0',inplace=True)

In [36]:
union_bank_df.head()

,Date,Description/Location,Reference,Amount
0,9/7,MET LIFE INS. CO HCCLAIMPMT CCD 1ML000416513,51062176.0,258.00
1,9/8,MET LIFE INS. CO HCCLAIMPMT CCD 1ML000449313,54941449.0,62.00
2,9/9,MET LIFE INS. CO HCCLAIMPMT CCD 1ML000498167,54943552.0,159.00
3,9/9,MET LIFE INS. CO HCCLAIMPMT CCD 1ML000482103,54943445.0,"2,256.80"
4,9/12,MET LIFE INS. CO HCCLAIMPMT CCD 1ML000510443,57551411.0,99.00


### Use my function to extract transaction

In [37]:
union_dir ='C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/Union Bank.pdf'
union_df = convert_pdf_to_csv(union_dir)

Connect file successful
File Union Bank.pdf: 40 transactions




In [39]:
union_df['Union Bank']['DataFrame'].sample(10,random_state=26)

,Transaction,Trans_date,Post_date,Amount,Description
7,9/14 MET LIFE INS. CO HCCLAIMPMT CCD 1ML000591735 58829227 228.00,,9/14,228.00,MET LIFE INS. CO HCCLAIMPMT CCD 1ML000591735 58829227
11,9/16 MET LIFE INS. CO HCCLAIMPMT CCD 1ML000635843 53023532 297.00,,9/16,297.00,MET LIFE INS. CO HCCLAIMPMT CCD 1ML000635843 53023532
28,"9/1 TADS TUITION AID TADS PMNT PPD ***********7592 59567359 $ 1,379.13",,9/1,"1,379.13",TADS TUITION AID TADS PMNT PPD ***********7592 59567359 $
5,9/12 MET LIFE INS. CO HCCLAIMPMT CCD 1ML000510443 57551411 99.00,,9/12,99.00,MET LIFE INS. CO HCCLAIMPMT CCD 1ML000510443 57551411
27,9/30 MET LIFE INS. CO HCCLAIMPMT CCD 1ML001020286 54727225 536.00,,9/30,536.00,MET LIFE INS. CO HCCLAIMPMT CCD 1ML001020286 54727225
38,9/27 ATT*BILL P 800-288-2020 TX 800-288-2020 TX 73051106 454.05,,9/27,454.05,ATT*BILL P 800-288-2020 TX 800-288-2020 TX 73051106
33,"9/22 AMERICAN FUNDS INVESTMENT CCD 1,922.88",,9/22,"1,922.88",AMERICAN FUNDS INVESTMENT CCD
8,"9/14 MET LIFE INS. CO HCCLAIMPMT CCD 1ML000573158 58829108 1,294.00",,9/14,"1,294.00",MET LIFE INS. CO HCCLAIMPMT CCD 1ML000573158 58829108
9,9/15 MET LIFE INS. CO HCCLAIMPMT CCD 1ML000605526 50266292 292.80,,9/15,292.80,MET LIFE INS. CO HCCLAIMPMT CCD 1ML000605526 50266292
36,9/1 VZWRLSS*AP 800-922-0204 FL 800-922-0204 FL 70195694 $ 468.57,,9/1,468.57,VZWRLSS*AP 800-922-0204 FL 800-922-0204 FL 70195694 $


## Chase_check

### Get Deposit and Debts

In [40]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/Chase_Check.pdf'
chase_df = convert_pdf_to_csv(_dir)

Connect file successful
File Chase_Check.pdf: 17 transactions




In [42]:
chase_df['Chase_Check']['DataFrame'].sample(10,random_state=26)

,Transaction,Trans_date,Post_date,Amount,Description
9,"09/16 Orig CO Name:ADP Wage Pay Orig ID:9333006057 Desc Date:220916 CO Entry 1,585.04",,09/16,"1,585.04",Orig CO Name:ADP Wage Pay Orig ID:9333006057 Desc Date:220916 CO Entry
7,09/09 Orig CO Name:ADP Payroll Fees Orig ID:9659605001 Desc Date:220909 CO Entry 53.74,,09/09,53.74,Orig CO Name:ADP Payroll Fees Orig ID:9659605001 Desc Date:220909 CO Entry
8,"09/16 Orig CO Name:ADP - Tax Orig ID:9333006057 Desc Date:220916 CO Entry 1,785.87",,09/16,"1,785.87",Orig CO Name:ADP - Tax Orig ID:9333006057 Desc Date:220916 CO Entry
11,"09/30 Orig CO Name:ADP - Tax Orig ID:9333006057 Desc Date:220930 CO Entry 1,573.25",,09/30,"1,573.25",Orig CO Name:ADP - Tax Orig ID:9333006057 Desc Date:220930 CO Entry
13,"09/02 6,011.93",,09/02,"6,011.93",
12,"09/01 $8,028.03",,09/01,"8,028.03",
14,"09/06 5,470.03",,09/06,"5,470.03",
5,"09/01 Orig CO Name:ADP Wage Pay Orig ID:9333006057 Desc Date:220901 CO Entry 1,585.04",,09/01,"1,585.04",Orig CO Name:ADP Wage Pay Orig ID:9333006057 Desc Date:220901 CO Entry
2,"09/14 Online Transfer From Chk ...2112 Transaction#: 15295364896 7,500.00",,09/14,"7,500.00",Online Transfer From Chk ...2112 Transaction#: 15295364896
4,"09/01 Orig CO Name:ADP - Tax Orig ID:9333006057 Desc Date:220901 CO Entry $1,979.83",,09/01,"1,979.83",Orig CO Name:ADP - Tax Orig ID:9333006057 Desc Date:220901 CO Entry


 - **Missing one transaction in head of page** `09/23 52.23`

In [43]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/Chase_Check.pdf'
chase_checkno = get_check_no(_dir)

Connect file successful
File Chase_Check.pdf: 30 Checks


In [44]:
chase_checkno

{'Chase_Check': {'Status': '',
  'Transaction': 'File Chase_Check.pdf: 30 Checks',
  'Dataframe': ''}}

## Chase No check

In [45]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/Chase_No Check.pdf'
chase_df = convert_pdf_to_csv(_dir)

Connect file successful
File Chase_No Check.pdf: 68 transactions




In [46]:
chase_df['Chase_No Check']['DataFrame'].sample(10,random_state=26)

,Transaction,Trans_date,Post_date,Amount,Description
57,"09/15 09/14 Payment To Chase Card Ending IN 7014 1,132.15",,09/15,"1,132.15",09/14 Payment To Chase Card Ending IN 7014
11,"09/06 Orig CO Name:Not Just Snacks, Orig ID:945440567 Desc Date: CO Entry 61.52",,09/06,61.52,"Orig CO Name:Not Just Snacks, Orig ID:945440567 Desc Date: CO Entry"
63,"09/06 2,804.20",,09/06,"2,804.20",
2,09/02 Orig CO Name:Merch Bnkcd Nsd Orig ID:B526123456 Desc Date:220901 CO Entry 450.87,,09/02,450.87,Orig CO Name:Merch Bnkcd Nsd Orig ID:B526123456 Desc Date:220901 CO Entry
5,"09/02 Orig CO Name:Not Just Snacks, Orig ID:945440567 Desc Date: CO Entry 39.02",,09/02,39.02,"Orig CO Name:Not Just Snacks, Orig ID:945440567 Desc Date: CO Entry"
67,"09/12 3,311.65",,09/12,"3,311.65",
50,09/30 Orig CO Name:Grubhub Inc Orig ID:1261328194 Desc Date: CO Entry 41.37,,09/30,41.37,Orig CO Name:Grubhub Inc Orig ID:1261328194 Desc Date: CO Entry
36,"09/21 Online Transfer From Chk ...6553 Transaction#: 15350173428 15,000.00",,09/21,"15,000.00",Online Transfer From Chk ...6553 Transaction#: 15350173428
22,09/13 Orig CO Name:Merch Bnkcd Nsd Orig ID:B526123456 Desc Date:220912 CO Entry 515.85,,09/13,515.85,Orig CO Name:Merch Bnkcd Nsd Orig ID:B526123456 Desc Date:220912 CO Entry
9,09/06 Orig CO Name:Merch Bnkcd Nsd Orig ID:B526123456 Desc Date:220905 CO Entry 574.10,,09/06,574.10,Orig CO Name:Merch Bnkcd Nsd Orig ID:B526123456 Desc Date:220905 CO Entry


- After testing, missing 1 transaction in head of paper `09/28 272.61`

## Well Fargo Credit Card

In [47]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/WF bank - citi bank/WF_CC 1.pdf'
well_cc_df = convert_pdf_to_csv(_dir)

Connect file successful
File WF_CC 1.pdf: 82 transactions




In [49]:
well_cc_df['WF_CC 1']['DataFrame'].head()

,Transaction,Trans_date,Post_date,Amount,Description
0,07/2507/272469216JZ2XXWZK4D THE HOME DEPOT 1840 LAKE FOREST CA 420.07,07/27,07/25,420.07,2469216JZ2XXWZK4D THE HOME DEPOT 1840 LAKE FOREST CA
1,"07/2607/272494300K0S4AFL4QE COSTCO WHSE #0454 IRVINE CA 1,486.87",07/27,07/26,"1,486.87",2494300K0S4AFL4QE COSTCO WHSE #0454 IRVINE CA
2,07/2707/272403454K102Y24SYP ARCO #42381 AMPM IRVINE CA 67.45,07/27,07/27,67.45,2403454K102Y24SYP ARCO #42381 AMPM IRVINE CA
3,07/2707/272443106K12DYTWBYM AMAZON.COM*3W8HW3B53 AMZN AMZN.COM/BILL WA 20.62,07/27,07/27,20.62,2443106K12DYTWBYM AMAZON.COM*3W8HW3B53 AMZN AMZN.COM/BILL WA
4,07/2707/272469216K02XJ4B6PV AMZN Mktp US*JA4V27PI3 Amzn.com/bill WA 37.66,07/27,07/27,37.66,2469216K02XJ4B6PV AMZN Mktp US*JA4V27PI3 Amzn.com/bill WA


## Well Fargo Checking

In [51]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/WF bank - citi bank/WF_Checking_1.pdf'
well_checking_df = convert_pdf_to_csv(_dir)

Connect file successful
File WF_Checking_1.pdf: 26 transactions




In [52]:
well_checking_df['WF_Checking_1']['DataFrame'].head()

,Transaction,Trans_date,Post_date,Amount,Description
0,"8/18 Corpuschristi 35 ACH Pmt 220818 5348774980 Aug 2022 Loan 1,765.56",,8/18,"1,765.56",Corpuschristi 35 ACH Pmt 220818 5348774980 Aug 2022 Loan
1,"8/18 Baytown Jacinto ACH Pmt 220818 5348775772 Aug 2022 Loan 1,962.44",,8/18,"1,962.44",Baytown Jacinto ACH Pmt 220818 5348775772 Aug 2022 Loan
2,"8/18 Sanantonio Conna ACH Pmt 220818 5348774758 Aug 2022 4,079.25",,8/18,"4,079.25",Sanantonio Conna ACH Pmt 220818 5348774758 Aug 2022
3,"8/18 Daymark Mesa Par ACH Pmt 220818 5348773488 Aug 2022 4,723.00",,8/18,"4,723.00",Daymark Mesa Par ACH Pmt 220818 5348773488 Aug 2022
4,"8/18 Valens Hyland PA ACH Pmt 220818 5348773744 Aug 2022 4,723.00",,8/18,"4,723.00",Valens Hyland PA ACH Pmt 220818 5348773744 Aug 2022


## Citi Checking

In [53]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/WF bank - citi bank/Citi_checking_1.pdf'
citi_checking_df = citi_bank_to_csv(_dir)

Connect file successful
File Citi_checking_1.pdf: 199 transactions




In [55]:
citi_checking_df['Citi_checking_1']['Dataframe'].head()

,Date,Description,Sub_description_1,Sub_description_2,Amount
0,08/01,ELECTRONIC CREDIT,MERCHANT BANKCD DEPOSIT 496428777882 Aug 01,,35.60
1,08/01,ELECTRONIC CREDIT,COMPREHENSIVE FI PAYMENT C21809 Aug 01,001/R1/20F013,298.69
2,08/01,ELECTRONIC CREDIT,MERCHANT BANKCD DEPOSIT 496428777882 Aug 01,,"3,082.50"
3,08/01,DEBIT CARD PURCH Card Ending in 0188,BGMM5LLB 000188 Aug 01,JERSEY MIKES 17014 HENDERSON NV 22210,18.34
4,08/01,DEBIT CARD PURCH Card Ending in 0188,K7S8Y0V3 000188 Aug 01,KANGS KITCHEN HENDERSON NV 22210,183.60


## Bank of Hope

In [269]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Bank of Hope Checking 0933 2022/Checking Account Statements - 10_31_2022 - 6400230933 - DULCE INC - 11_1_2022.pdf'
bank_hope_df = hope_bank_to_csv(_dir)

Checking Account Statements - 10_31_2022 - 6400230933 - DULCE INC - 11_1_2022: Connect file successful
File Checking Account Statements - 10_31_2022 - 6400230933 - DULCE INC - 11_1_2022.pdf: 106 transactions




In [270]:
bank_hope_df['Checking Account Statements - 10_31_2022 - 6400230933 - DULCE INC - 11_1_2022']['Dataframe'].head(50)

,Transaction,Trans_date,Post_date,Amount,Description
0,"10/01/2022 Beginning Balance $200,795.35",,10/01/2022,"200,795.35",Beginning Balance
1,"10/31/2022 Ending Balance $203,509.68",,10/31/2022,"203,509.68",Ending Balance
2,10/03/2022 STRIPE CN 07593 ST-B7L4Y2O0D5E9 $19.62,,10/03/2022,19.62,STRIPE CN 07593 ST-B7L4Y2O0D5E9
3,10/03/2022 STRIPE CN 07593 ST-I1M8Z7A4G1U7 $22.98,,10/03/2022,22.98,STRIPE CN 07593 ST-I1M8Z7A4G1U7
4,10/03/2022 STRIPE CN 07593 ST-I6H0C5A3Z8T8 $41.54,,10/03/2022,41.54,STRIPE CN 07593 ST-I6H0C5A3Z8T8
5,10/03/2022 STRIPE CN 07593 ST-M9P3S0B0T9Y0 $42.58,,10/03/2022,42.58,STRIPE CN 07593 ST-M9P3S0B0T9Y0
6,10/03/2022 STRIPE CN 07593 ST-O0A9O8E3Q8R4 $109.23,,10/03/2022,109.23,STRIPE CN 07593 ST-O0A9O8E3Q8R4
7,10/03/2022 Square Inc 221003P2 L209753897694 $200.21,,10/03/2022,200.21,Square Inc 221003P2 L209753897694
8,10/03/2022 STRIPE CN 07593 ST-A0E0Q0E9G4B4 $255.37,,10/03/2022,255.37,STRIPE CN 07593 ST-A0E0Q0E9G4B4
9,10/03/2022 STRIPE CN 07593 ST-S1N4O5E4L5U3 $259.07,,10/03/2022,259.07,STRIPE CN 07593 ST-S1N4O5E4L5U3


# Covert a folder which contains many pdf to excel file 

## Test Fucntion:

In [17]:
folder_pdf = r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Test'
folder_pdf_w = 'C:/Users/trilnd/Desktop/QBO/'
folder_excel = r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Test'
kind_function = 1

In [18]:
list_dfs = convert_pdffolder_to_excelfolder(pdf_path=folder_pdf,
                                            xlsx_path=folder_excel,
                                            function_kind=kind_function)

022822 WellsFargo: Connect file successful
File 022822 WellsFargo.pdf: 65 transactions


082322 WellsFargo: Connect file successful


Xref table not zero-indexed. ID numbers for objects will be corrected.


File 082322 WellsFargo.pdf: 100 transactions


2022.08: Connect file successful
File 2022.08.pdf: 84 transactions


20220817-statements-1475-: Connect file successful
20220822-statements-2904-: Connect file successful


In [101]:
#print(list_dfs)

## Breakdown function:

In [65]:
pdf_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/'
xlsx_path = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/'
list_file = [os.path.join(pdf_dir,f) for f in os.listdir(pdf_dir) if f.endswith('.pdf')]
list_file

['C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.01.pdf',
 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.02.pdf',
 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.03.pdf',
 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.04.pdf',
 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.05.pdf',
 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.06.pdf',
 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.07.pdf',
 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.08.pdf',
 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.09.pdf',
 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.10.pdf']

In [67]:
function_kind = 1
function_list = [convert_pdf_to_csv,
                 citi_bank_to_csv,
                 get_check_no,
                 hope_bank_to_csv]
function = function_list[function_kind-1]
list_pdfs = []

In [61]:
function == get_check_no

False

In [62]:
len(list_file)

10

In [68]:
if len(list_file) == 0:
    print("There is no file PDF. Please choose again")
    #return "There is no file PDF. Please choose again"
else:
    if function != get_check_no:
        list_file = [file for file in list_file if
                     os.path.exists(os.path.join(xlsx_path, os.path.basename(file)[:-4] + '.xlsx')) is False]
        for file in list_file:
            try:
                dict_pdf = function(pdf_path=file, excel_path=xlsx_path)
            except Exception:
                continue
            else:
                list_pdfs.append(dict_pdf)
    else:
        for file in list_file:
            try:
                dict_pdf = get_check_no(pdf_path=file, excel_path=xlsx_path)
            except Exception:
                continue
            else:
                list_pdfs.append(dict_pdf)

Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


2022.02: Connect file successful
File 2022.02.pdf: 83 transactions


2022.03: Connect file successful


Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


File 2022.03.pdf: 112 transactions


2022.04: Connect file successful
File 2022.04.pdf: 103 transactions


2022.05: Connect file successful


Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


...

File 2022.07.pdf: 119 transactions


2022.08: Connect file successful


Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


File 2022.08.pdf: 123 transactions


2022.09: Connect file successful


Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


File 2022.09.pdf: 121 transactions


2022.10: Connect file successful
File 2022.10.pdf: 112 transactions




In [47]:
list_output[1]

{'2022.02': {'Status': 'Connect file successful',
  'Transaction': 'File 2022.02.pdf: 83 transactions',
  'Dataframe':                                                                          Transaction Trans_date Post_date      Amount                                                          Description
  0                     02/01 Beginning Balance $97,738.05 Number of Days in Period 28                02/01   97,738.05                        alance $97,738.05 Number of Days in Period 28
  1                                                   02/28 Ending Balance $108,505.11                02/28  108,505.11                                                       ce $108,505.11
  2   02/01 149.89 QUILL CORPORATIO QUILL BT0131 000000165236681 Alexander Choe 020122                02/01      149.89  QUILL CORPORATIO QUILL BT0131 000000165236681 Alexander Choe 020122
  3   02/01 13,605.03 WEB INITIATED PAYMENT AT CHASE CREDIT CRD EPAY 5754112239 020122                02/01   13,605.03     WEB

In [49]:
[info['Status'] for dict_pdf in list_output for (key,info) in dict_pdf.items()]

['Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful']

In [50]:
status_pdf = [info['Status'] for dict_pdf in list_output for (key,info) in dict_pdf.items()]
trans_pdf = [info['Transaction'] for dict_pdf in list_output for (key,info) in dict_pdf.items()]
 #if isinstance(info['DataFrame'], pd.DataFrame)]

In [51]:
status_pdf

['Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful',
 'Connect file successful']

In [52]:
message = '\n'.join([f'{s}\n{t}\n' for s,t in zip(status_pdf,trans_pdf)])
print(message)

Connect file successful
File 2022.01.pdf: 59 transactions

Connect file successful
File 2022.02.pdf: 83 transactions

Connect file successful
File 2022.03.pdf: 112 transactions

Connect file successful
File 2022.04.pdf: 103 transactions

Connect file successful
File 2022.05.pdf: 97 transactions

Connect file successful
File 2022.06.pdf: 122 transactions

Connect file successful
File 2022.07.pdf: 119 transactions

Connect file successful
File 2022.08.pdf: 123 transactions

Connect file successful
File 2022.09.pdf: 121 transactions

Connect file successful
File 2022.10.pdf: 112 transactions



In [54]:
valid_dfs = [ info['Dataframe'] 
             for dict_pdf in list_output 
             for (key,info) in dict_pdf.items() 
             if isinstance(info['Dataframe'], pd.DataFrame)]

In [55]:
len(valid_dfs)

10

In [56]:
sum([df.shape[0] for df in valid_dfs])

1051

## try the PDF with scan data

In [93]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/File scan/P. Putrasahan DDS Inc.pdf'
convert_pdf_to_csv(_dir)

P. Putrasahan DDS Inc: Connect file successful
This PDF file is Scan data


{'P. Putrasahan DDS Inc': {'Status': 'This PDF file is Scan data',
  'Transaction': '',
  'Dataframe': ''}}

# Breakdown the Function to keep tracking ouput easier 

## MY FUNCTION

In [10]:
def get_description(trans,len_upper,len_lower):
    if re.search('www.53.com/businessbanking',all_text_page):
        return trans[len_upper+len_lower+1:]
    else:
    # check space: 
        check_space = pat+'(?=\s)'
        if re.search(check_space,trans) != None:
            return trans[len_upper+1:(len(trans)-len_lower)]
        else:
            return trans[(len_upper):(len(trans)-len_lower)]

- ***IMPORTANT PARAMETER***

In [11]:
# pdf_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder'

# list_file = [os.path.join(pdf_dir,f) for f in os.listdir(pdf_dir) if f.endswith('.pdf')]
# list_file

In [66]:
pdf_path = r'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Capital One CC 7682 (0036) 2022/2022.02.16.pdf'
# r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\AMEX CC 53003 2022\2022-10-10.pdf'
# r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Bank of the West Checking 8318 2022\Statement 02_28_22.pdf'
# r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Bank of the West CC 5867 202\2022.02.28.pdf'
# r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Test\082322 WellsFargo.pdf'
# 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fifth Third Bank 6333 2022/2022.04.pdf' 
# 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/Checking e-Statement - 10_31_2022 - IRIFF ULEP DDS INC.pdf' 
# 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/2022.09.30.pdf'
#'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/BUSINESS CARD (...6959) - chase.com.pdf' 
# 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Citi CC.pdf'
# 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/2022-10-28 AMEX.pdf'
# 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/2022.08 SignatureChecking.pdf'
#'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/American_2y.pdf'
#'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/20220831-statements-7227-.pdf'

#'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/2022.08.pdf'
#'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/credit_card.pdf'
#'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/AMEX CC.pdf'
#'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/WF_CC 1.pdf'

#'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/American_2y.pdf'
#'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/082322 WellsFargo.pdf'
#'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/022822 WellsFargo.pdf'
# 'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/BofA.pdf'
#'C:/Users/trilnd/Desktop/QBO/File PDF/Bank Statement/WF bank - citi bank/WF_CC 1.pdf'
base_name = os.path.basename(pdf_path)[0:-4]
dict_info = {base_name:{}}
base_name
#kind_extract_pattern = 2

'2022.02.16'

In [67]:
dict_info

{'2022.02.16': {}}

In [86]:
pattern_1 = r"^([A-Z][a-z]{2}\s?\d{1,2})"
pattern_2 = r'^((\d{1,2}/\d{1,2})(/\d{2,4})?)'
pattern_3 = r'^((\d{1,2}-\d{1,2})(-\d{2,4})?)'


#1: Check whether the path is correct or wrong
try:
    with open(pdf_path,'rb') as file:
        print("Connect file successful")
except IOError:
        print("Can not find the file directory or file name does not exist")

#2: Implement to convert file (Beautiful Transaction) to pdf variable and print total number pages
else:
    try:
        pdf_file = PdfReader(pdf_path,strict=True)
    except Exception as e:
        raise e
    else:
        number_page = len(pdf_file.pages)
        print("Total number pages of File PDF are",number_page)


Connect file successful


PdfReadError: PDF starts with '
Con', but '%PDF-' expected

In [73]:
pdf_file.isEncrypted

False

In [74]:
#3: Convert pdf file to the string 
all_text_page =''

for page_num in range(0,pdf_file.getNumPages()):
    try :
        text = pdf_file.getPage(page_num).extract_text()
    except PdfReadError as e:
        print(e)
    else:
        all_text_page += (text+'\n')

if re.search(r'.',all_text_page) == None:
    raise Exception("This file without text data")
else:
    all_text_list = [line.strip() for line in all_text_page.split('\n')]

KeyError: '/Root'

In [65]:
all_text_list

['001/R1/20F013',
 'Citibank CBO Services',
 '715',
 'P.O. Box 6201',
 '003',
 'Sioux Falls, SD 57117-6201',
 'CITIBANK, N. A.',
 'Account',
 '207481896',
 'Statement Period',
 'MR PATEL DDS, INC',
 'Jan 1 - Jan 31, 2022',
 'SIXTH & SMITH DENTAL CARE',
 '1530 W. 6TH STREET Suite 105',
 'Relationship Manager',
 'CORONA                CA 92882',
 '',
 'US SERVICE CENTER',
 '1-877-528-0990',
 'Page 1  of  3',
 '®',
 'CitiBusiness',
 'ACCOUNT AS OF JANUARY 31, 2022',
 'Relationship Summary:',
 'Checking',
 '$7,813.43',
 'Savings',
 '-----',
 'Checking Plus',
 '-----',
 'SERVICE CHARGE SUMMARY FROM DECEMBER 1, 2021 THRU DECEMBER 31, 2021',
 'Type of Charge',
 'No./Units',
 'Price/Unit',
 'Amount',
 'STREAMLINED CHECKING # 207481896',
 'Average Daily Collected Balance',
 '$5,554.43',
 'DEPOSIT SERVICES',
 'CURRENCY DEPOSIT (PER $100)',
 '99',
 '.1700',
 '16.83',
 '**WAIVE',
 'CHECKS, DEP ITEMS/TICKETS, ACH',
 '36',
 '.4500',
 '16.20',
 '**WAIVE',
 'Total Charges for Services',
 '$0.00',
 'Ne

In [74]:
#print(all_text_page)

In [55]:
if re.search(r'.',all_text_page) == None:
    raise Exception("This is the file with scan data")
else:
    print('Success')

Success


In [186]:
#all_text_list

In [56]:
# Select amount pattern if this PDF is AMEX or Citi => amount contains $
if re.search('www.americanexpress.com',all_text_page) or re.search('www.citicards.com',all_text_page):
    num_pat = r'(-?\s?\$\d{1,3}(?:,\d{3})*\.\d{2})'
else:
    num_pat = r'(-?\s?\$?\d{1,3}(?:,\d{3})*\.\d{2})'

In [57]:
#4 Select date pattern
count_p1 = sum([1 for line in all_text_list if re.search(pattern_1,line)])
count_p2 = sum([1 for line in all_text_list if re.search(pattern_2,line)])
count_p3 = sum([1 for line in all_text_list if re.search(pattern_3,line)])
 
if count_p1 > count_p2 and count_p1 > count_p3:
    kind_extract_pattern = 1
    pat = pattern_1
    print("Chose the patern 1")

elif count_p2 > count_p1 and count_p2 > count_p3:
    kind_extract_pattern = 2
    pat = pattern_2
    print("Chose the patern 2")
    
elif count_p3 > count_p1 and count_p3 > count_p2:
    kind_extract_pattern = 2
    pat = pattern_3
    print("Chose the patern 3")  

else:
    print("ERROR: There is not proper pattern for this file")

Chose the patern 2


In [58]:
len(all_text_list)

298

In [60]:
#all_text_list

In [61]:
# Search Bank of west CC
extract_list = []
if re.search('www.bankofthewestcorporaterewards.com',all_text_page.replace('\n','')):
    num_pat_BOW = r'(^\d{1,3}(?:,\d{3})*\.\d{2}$)'
    for i in range(len(all_text_list)):
        try:
            line = all_text_list[i]
            next_line = all_text_list[i+1]
            if re.compile(pattern=pat).search(line) and re.compile(pattern=pat).search(next_line):
                transaction = all_text_list[i:i+2]

            if re.search(pattern=num_pat_BOW,string=next_line):
                amount = [line,next_line]
                transaction.extend(amount)
                line = ' '.join(transaction)
                extract_list.append(line)
        except IndexError:
            break
else:
    for i in range(len(all_text_list)):
        line = all_text_list[i]

        if re.search(pat,line) and re.search(num_pat,line):
            extract_list.append(line.strip())

        elif re.search(pat,line):
            extract_list.append(line)

        elif re.search(num_pat,line):
            span = re.search(num_pat,line).span()
            only_numeric = line[span[0]:span[1]]
            extract_list.append(only_numeric)

In [64]:
#extract_list

In [159]:
pat

'^((\\d{1,2}-\\d{1,2})(-\\d{2,4})?)'

In [160]:
num_pat

'(^\\d{1,3}(?:,\\d{3})*\\.\\d{2}$)'

In [164]:
# 6: GET ALL TRANSACTION START WITH DATE OR COMBINE ROW (START WITH DATE) AND ROW NUMBER
filter_list = []
if re.search('www.53.com/businessbanking',all_text_page):
    for i in range(len(extract_list)):
        line = extract_list[i]
        if re.search(pat,line) and re.search(num_pat,line):
            filter_list.append(line.strip())

else:
    for i in range(len(extract_list)):
        line = extract_list[i]
        # if this line contains both of date and amount => append to filter list
        if re.search(pat,line) and re.search(num_pat,line):
            span = re.search(num_pat,line).span()
            line = line[0:span[1]]
            filter_list.append(line.strip())

        # if this line is date = > combine with next line and append to next list
        elif re.search(pat,line):
            next_line = extract_list[i+1]

            # if next line is date get next second line
            if re.search(pat,next_line) != None:
                new_line = line +' '+ extract_list[i+2]
                filter_list.append(new_line)
            else:
                new_line = line +' '+ next_line
                filter_list.append(new_line)

        # if this line is number: => continue 
        elif re.search(num_pat,line):
            continue

In [165]:
filter_list

['02-22 02-22 PRE-AUTHORIZED PAYMENT 5,784.42',
 '01-31 01-27 MAURICE CARRIE WINERY    TEMECULA     CA 89.93',
 '01-31 01-27 BOARD AND BREW TEMECUL   TEMECULA     CA 129.85',
 '01-31 01-30 SPECTRUM                 855-707-7328 MO 199.99',
 '01-31 01-31 RUBIOS 045 ECOM          858-613-7785 CA 25.24',
 '02-07 02-04 Google ADS4991507970     Mountain ViewCA 349.72',
 '02-07 02-04 PAYPAL *EVO              4029357733   WA 258.55',
 '02-10 02-10 NET32* PA DENTAL         CARY         NC 27.17',
 '02-11 02-10 NET32* RIVER DENTAL SU   CARY         NC 40.18',
 '02-11 02-10 NET32* FRONTIER DENTAL   CARY         NC 37.53',
 '02-11 02-10 NET32* MVP DENTAL SUPP   CARY         NC 189.19',
 '02-11 02-10 URBANE CAFE              9512966251   CA 155.83',
 '02-14 02-11 NET32* TRADENT SUPPLY    CARY         NC 64.72',
 '02-14 02-13 Disney PLUS              Burbank      CA 7.32',
 '02-15 02-14 SRFAX                    604-7138000  BC 9.95',
 '02-15 02-15 INTERNATIONAL TRANSACTION FEE 0.25',
 '02-21 02-18 F

***#6: Convert list to dataFrame***

In [166]:
trans_df = pd.DataFrame(filter_list,columns=['Transaction'])

# Get only transaction which contaims amount
trans_df = trans_df[~trans_df.Transaction.apply(lambda x: re.search(num_pat,x)).isnull()].reset_index(drop=True)

# Print total transaction in file pdf
status= f"Total transaction in file PDF: {trans_df.size}"
dict_info[base_name]["Status"] = status
print(status)

Total transaction in file PDF: 25


In [167]:
dict_info

{'2022.02.28': {'Status': 'Total transaction in file PDF: 25'}}

In [168]:
kind_extract_pattern

2

In [169]:
# Create 2 columns Post date and Trans date

if kind_extract_pattern == 2:
    
    # Define pattern
    kind_2_pat_2d = r'(\d{1,2}[/-]\d{1,2})\s?(\d{1,2}[/-]\d{1,2})'
    kind_2_pat_1d = r'(\d{1,2}[/-]\d{1,2})'
    kind_2_pat_has_year = r'(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
    
    # Create filter 
    has_year = trans_df.Transaction.str.contains(pat=kind_2_pat_has_year).sum()/trans_df.shape[0]
    has_2date = trans_df.Transaction.str.contains(pat=kind_2_pat_2d).sum()/trans_df.shape[0]
    
    # Check case 1: Has year ?
    if has_year > 0.7:
        trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_has_year)
        trans_df['Trans_date'] = ['']*trans_df.shape[0]
    
    # Check case 2: Has two date? Post date and Trans date
    elif has_2date > 0.7:
        trans_df[['Post_date','Trans_date']] = trans_df.Transaction.str.extract(pat=kind_2_pat_2d)
        
        # If there are some transaction having 1 date => get this date to Post day
        outlier = (trans_df['Transaction'].str.contains(pat=kind_2_pat_2d) == False)
        trans_df.iloc[outlier,[2]] =trans_df[outlier].Transaction.str.extract(pat=kind_2_pat_1d)
        
    else:
        trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_1d)
        trans_df['Trans_date'] = ['']*trans_df.shape[0] 

if kind_extract_pattern == 1:
    # Define pattern
    kind_1_pat_2d = r'([A-Z][a-z]{2}\s?\d{1,2})\s?([A-Z][a-z]{2}\s?\d{1,2})'
    kind_1_pat_1d = r'([A-Z][a-z]{2}\s?\d{1,2})'
    kind_1_pat_has_year = r'([A-Z][a-z]{2}\s?\d{1,2},?\s?\d{2,4})'
    
    # Create filter 
    has_2date = trans_df.Transaction.str.contains(pat=kind_1_pat_2d).sum()/trans_df.shape[0]
    has_year = trans_df.Transaction.str.contains(pat=kind_1_pat_has_year).sum()/trans_df.shape[0]
    
    # Check case 1: Has year ?
    if has_year > 0.7:
        trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_1_pat_has_year)
        trans_df['Trans_date'] = ['']*trans_df.shape[0]
    
    # Check case: Has two date? Post date and Trans date
    elif has_2date > 0.7:
        trans_df[['Post_date','Trans_date']] = trans_df.Transaction.str.extract(pat=kind_1_pat_2d)
        
        # If there are some transaction having 1 date => get this date to Post day
        outlier = (trans_df['Transaction'].str.contains(pat=kind_2_pat_2d) == False)
        trans_df.iloc[outlier,[2]] = trans_df[outlier].Transaction.str.extract(pat=kind_1_pat_1d)
        
    else:
        trans_df['Post_date'] = trans_df.Transaction.str.extract(pat=kind_2_pat_1d)
        trans_df['Trans_date'] = ['']*trans_df.shape[0] 


In [170]:
trans_df.head()

,Transaction,Post_date,Trans_date
0,"02-22 02-22 PRE-AUTHORIZED PAYMENT 5,784.42",02-22,02-22
1,01-31 01-27 MAURICE CARRIE WINERY TEMECULA CA 89.93,01-31,01-27
2,01-31 01-27 BOARD AND BREW TEMECUL TEMECULA CA 129.85,01-31,01-27
3,01-31 01-30 SPECTRUM 855-707-7328 MO 199.99,01-31,01-30
4,01-31 01-31 RUBIOS 045 ECOM 858-613-7785 CA 25.24,01-31,01-31


In [171]:
trans_df = trans_df.fillna(value='')

In [172]:
# Create Amount columns     
trans_df['Amount'] = trans_df.Transaction.str.extract(pat=num_pat,expand=False).str.replace(pat=r'[$\s]',repl='',regex=True)

# Create the Description columns
# len upper will prase total len of the date which is not included the space
trans_df['len_date']=trans_df[['Trans_date','Post_date']].apply(lambda x: len(x['Trans_date'])+len(x['Post_date']), axis=1)

trans_df['len_amount'] = trans_df.Transaction.str.extract(num_pat,expand=False).apply(lambda x: len(x))

trans_df['Description'] = trans_df.apply(lambda x: get_description(trans=x['Transaction'],
                                                                   len_upper=x['len_date'],
                                                                   len_lower=x['len_amount']),axis=1)

In [173]:
trans_df.head(50)

,Transaction,Post_date,Trans_date,Amount,len_date,len_amount,Description
0,"02-22 02-22 PRE-AUTHORIZED PAYMENT 5,784.42",02-22,02-22,"5,784.42",10,9,PRE-AUTHORIZED PAYMENT
1,01-31 01-27 MAURICE CARRIE WINERY TEMECULA CA 89.93,01-31,01-27,89.93,10,6,MAURICE CARRIE WINERY TEMECULA CA
2,01-31 01-27 BOARD AND BREW TEMECUL TEMECULA CA 129.85,01-31,01-27,129.85,10,7,BOARD AND BREW TEMECUL TEMECULA CA
3,01-31 01-30 SPECTRUM 855-707-7328 MO 199.99,01-31,01-30,199.99,10,7,SPECTRUM 855-707-7328 MO
4,01-31 01-31 RUBIOS 045 ECOM 858-613-7785 CA 25.24,01-31,01-31,25.24,10,6,RUBIOS 045 ECOM 858-613-7785 CA
5,02-07 02-04 Google ADS4991507970 Mountain ViewCA 349.72,02-07,02-04,349.72,10,7,Google ADS4991507970 Mountain ViewCA
6,02-07 02-04 PAYPAL *EVO 4029357733 WA 258.55,02-07,02-04,258.55,10,7,PAYPAL *EVO 4029357733 WA
7,02-10 02-10 NET32* PA DENTAL CARY NC 27.17,02-10,02-10,27.17,10,6,NET32* PA DENTAL CARY NC
8,02-11 02-10 NET32* RIVER DENTAL SU CARY NC 40.18,02-11,02-10,40.18,10,6,NET32* RIVER DENTAL SU CARY NC
9,02-11 02-10 NET32* FRONTIER DENTAL CARY NC 37.53,02-11,02-10,37.53,10,6,NET32* FRONTIER DENTAL CARY NC


In [39]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Transaction  100 non-null    object
 1   Post_date    100 non-null    object
 2   Trans_date   100 non-null    object
 3   Amount       100 non-null    object
 4   len_date     100 non-null    int64 
 5   len_amount   100 non-null    int64 
 6   Description  100 non-null    object
dtypes: int64(2), object(5)
memory usage: 5.6+ KB


In [50]:
trans_df[['Transaction','Trans_date','Post_date','Description','Amount']].to_excel(f'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/Fix File/{base_name}.xlsx',index=False)

In [51]:
dict_info[base_name]["Dataframe"] = trans_df

In [52]:
dict_info[base_name]["Dataframe"]

,Transaction,Post_date,Trans_date,Amount,len_date,len_amount,Description
0,"04/01 Beginning Balance $113,419.18 Number of Days in Period 30",04/01,,"113,419.18",5,12,"lance $113,419.18 Number of Days in Period 30"
1,"04/30 Ending Balance $154,725.70",04/30,,"154,725.70",5,12,"ce $154,725.70"
2,04/04 76.13 MERCH BANKCARD BILLNG 520004511336 ALEXANDER CHOE DDS INC MERCH BANKCARD 520004511336,04/04,,76.13,5,6,MERCH BANKCARD BILLNG 520004511336 ALEXANDER CHOE DDS INC MERCH BANKCARD 520004511336
3,04/05 20.52 PAY PLUS PPSACCOUNT ACHTRANS 452579291 ZP Account 5 040522,04/05,,20.52,5,6,PAY PLUS PPSACCOUNT ACHTRANS 452579291 ZP Account 5 040522
4,"04/05 1,411.22 5/3 COMMRCL LN #XXXXXXXXXXX0018 PAID BY AUTO BILLPAYER",04/05,,"1,411.22",5,9,5/3 COMMRCL LN #XXXXXXXXXXX0018 PAID BY AUTO BILLPAYER
...,...,...,...,...,...,...,...
98,"04/05 118,686.03 04/14 138,164.65 04/25 149,808.67",04/05,,"118,686.03",5,11,"04/14 138,164.65 04/25 149,808.67"
99,"04/06 121,379.38 04/15 138,328.85 04/26 154,327.62",04/06,,"121,379.38",5,11,"04/15 138,328.85 04/26 154,327.62"
100,"04/07 131,075.52 04/18 142,718.05 04/27 159,769.53",04/07,,"131,075.52",5,11,"04/18 142,718.05 04/27 159,769.53"
101,"04/08 129,935.60 04/19 145,512.83 04/28 154,469.98",04/08,,"129,935.60",5,11,"04/19 145,512.83 04/28 154,469.98"


## CITI CHECKING

In [36]:
pdf_path = r'C:\Users\trilnd\Desktop\QBO\File PDF\Test PDF folder\Citi Checking 1896 2022\2022.01.pdf'
base_name = os.path.basename(pdf_path)[0:-4]
base_name
#kind_extract_pattern = 2

'2022.01'

In [37]:
pattern_1 = r"^([A-Z][a-z]{2}\s?\d{1,2})"
pattern_2 = r'^((\d{1,2}/\d{1,2})(/\d{2})?)'
num_pat = r'(-?\s?\$?\d+,?\d*\.\d*)'

#1: Check whether the path is correct or wrong
try:
    with open(pdf_path,'rb') as file:
        print("Connect file successful")
except IOError:
        print("Can not find the file directory or file name does not exist")

#2: Implement to convert file (Beautiful Transaction) to pdf variable and print total number pages
else:
    try:
        pdf_file = PdfReader(pdf_path,strict=True)
    except Exception as e:
        raise e
    else:
        number_page = pdf_file.getNumPages()
        print("Total number pages of File PDF are",number_page)


Connect file successful
Total number pages of File PDF are 4


In [38]:
#3: Convert pdf file to the string 

all_text_page =''

for page_num in range(0,pdf_file.getNumPages()):
    all_text_page += (pdf_file.getPage(page_num).extractText()+'\n')

all_text_list = [line.strip() for line in all_text_page.split('\n')]


In [40]:
#all_text_list

In [42]:
#4 Select pattern
count_p1 = sum([1 for line in all_text_list if re.search(pattern_1, line)])
count_p2 = sum([1 for line in all_text_list if re.search(pattern_2, line)])

if count_p1 > count_p2:
    kind_extract_pattern = 1
    pat = pattern_1
    print("Chose the patern 1")


elif  count_p1 < count_p2:
    kind_extract_pattern = 2
    pat = pattern_2
    print("Chose the patern 2")
else:
    print("ERROR: There is not proper pattern for this file")

Chose the patern 2


In [145]:
#all_text_list[68:100]

In [43]:
test_list = []
for i in range(len(all_text_list)):
    line = all_text_list[i]
    if re.search(pat,line):
        list_trans = [line for line in all_text_list[i:(i+6)] if line !='']
        test_list.append(list_trans)

In [44]:
for line in test_list:
    line[-1] = re.sub(pattern=pat,repl='',string=line[-1])

In [53]:
#test_list[:5]

***#6: Convert list to dataFrame***

In [45]:
trans_df = pd.DataFrame(test_list ,columns=['Date','Description','Amount','Balance','Sub_description_1','Sub_description_2'])

# Print total transaction in file pdftest_list
print(f"Total transaction in file PDF: {trans_df.shape[0]}")

Total transaction in file PDF: 36


In [46]:
trans_df

,Date,Description,Amount,Balance,Sub_description_1,Sub_description_2
0,01/03,ELECTRONIC CREDIT,"1,885.78","8,001.46",DCUSA DELTACA2D HCCLAIMPMT DC057602 Jan 03,TRN*1*202112210001742*1941461312*DDCA2~
1,01/04,TRANSFER CREDIT,"2,500.00","10,501.46",TRANSFER FROM CHECKING Jan 04,042027929597 VIA CBusOL Re # 048904
2,01/04,CHECK NO: 1162,"5,354.89","5,146.57",,None
3,01/06,ELECTRONIC CREDIT,"1,964.00","7,110.57",DXC TECHNOLOGY HCCLAIMPMT G868670122006 Jan 06,TRN*1*620023074*1271510177~
4,01/06,ACH DEBIT,"4,748.44","2,362.13",PAYROLL SERVICE 9F7R 9F7R 9F7R Jan 06,
5,01/07,ELECTRONIC CREDIT,3.00,"2,365.13",DCUSA DELTACA2D HCCLAIMPMT DC057602 Jan 07,TRN*1*202201040004090*1941461312*DDCA2~
6,01/10,ACH DEBIT,430.73,"1,934.40",SO CAL EDISON CO BILL PAYMT 700288404686 Jan 10,
7,01/11,ACH DEBIT,14.00,"1,920.40",CLOVER APP MRKT CLOVER APP 899-9335845-000 Jan 11,
8,01/12,ELECTRONIC CREDIT,413.60,"2,334.00",MERCHANT BNKCD DEPOSIT 334326558882 Jan 12,
9,01/12,ACH DEBIT,250.00,"2,084.00","CYNU,INC ACH Collec 1046020 Jan 12",001/R1/20F013


In [47]:
trans_df[['Description','Sub_description_1','Sub_description_2']].replace(regex=True,to_replace=r'\s+',value=' ',inplace=True)

In [48]:
trans_df.to_excel(excel_writer=f'C:/Users/trilnd/Desktop/QBO/Test_file/{base_name}.xlsx',index=False)

## USE TABULA TO EXTRACT THE TABLE FROM PDF FILE

In [201]:
_dir = 'C:/Users/trilnd/Desktop/QBO/File PDF/Test PDF folder/20220822-statements-2904-.pdf'

In [202]:
dfs = tabula.read_pdf(input_path=_dir,pages='all',stream=True)

In [203]:
len(dfs)

4

In [208]:
dfs[3].dropna(how='all').head(30)

,08/14,INDIGO LA DOWNTOWN LOS ANGELES CA,118.90
0,08/14,BUDGET RENT A CAR ONTARIO CA,178.98
1,08/13,PALM - SHC WESTERN INC LOS ANGELES CA,446.60
2,08/15,DIN TAI FUNG TIGARD PORTLAND OR,185.00
3,08/16,DOORDASH*JOY TERIYAKI WWW.DOORDASH. CA,1.63
4,08/15,OREGON DRIVERS EDUCATION 503-5813783 OR,65.00
5,08/14,INDIGO LA DOWNTOWN LOS ANGELES CA,23.24
6,08/14,INDIGO LA DOWNTOWN LOS ANGELES CA,23.24
7,08/16,FRED MEYER 0660 WOOD VILLAGE OR,7.99
8,08/15,LA CITY PARKING METER LOS ANGELES CA,2.00
9,08/16,SQ *VN CORNER VIETNAMESE Clackamas OR,28.75
